# About this notebook

@author: Yingding Wang\
@created: 28.Dec 2023\
@upated: 26.Feb 2024\
@version: 2

This notebook demonstrate the use of sqlalchemy client to connect to a postgres db.

* postgres client (psycopg): https://www.psycopg.org/
* Documentation (psycopg): https://www.psycopg.org/docs/usage.html
* Sqlalchemy: https://docs.sqlalchemy.org/en/20/
* use sqlalchemy with psycopg (3): https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg
* https://stackoverflow.com/questions/73596058/creating-an-sqlalchemy-engine-based-on-psycopg3/75885662#75885662

In [1]:
import sys, os, applyllm
# !{sys.executable} -m pip install --upgrade --use-feature=2020-resolver psycopg2-binary 2.9.9 python-dotenv==1.0.0
# !{sys.executable} -m pip install --user --upgrade psycopg[binary]==3.1.16 SQLAlchemy==2.0.24 python-dotenv==1.0.0

print(f"applyllm version: {applyllm.__version__}")

applyllm version: 0.0.3


## (Option 1) Edit postgres.env file for DB connection

In [2]:
# ENV_FILE="postgres.env"

### Uncomment the following cell and edit with your PostresDB credential to connect to DB

In [3]:
'''
%%writefile $ENV_FILE
# environment variables for Postgres DB 14.1 
DB_HOST="Database_Host_DNS_NAME"
DB_NAME="Database_Name"
DB_PORT="Database_Port"
DB_USER="Database_Login_User_Name"
DB_USER_PW="Database_Login_Password"
'''

'\n%%writefile $ENV_FILE\n# environment variables for Postgres DB 14.1 \nDB_HOST="Database_Host_DNS_NAME"\nDB_NAME="Database_Name"\nDB_PORT="Database_Port"\nDB_USER="Database_Login_User_Name"\nDB_USER_PW="Database_Login_Password"\n'

In [4]:
# from dotenv import load_dotenv
# load_dotenv(dotenv_path="postgres.env", override=True)

"""
print(f"\
{os.environ['DB_HOST']}\n\
{os.environ['DB_NAME']}\n\
{os.environ['DB_PORT']}\n\
{os.environ['DB_USER']}\n\
{os.environ['DB_USER_PW']}\n\
")
"""

'\nprint(f"{os.environ[\'DB_HOST\']}\n{os.environ[\'DB_NAME\']}\n{os.environ[\'DB_PORT\']}\n{os.environ[\'DB_USER\']}\n{os.environ[\'DB_USER_PW\']}\n")\n'

In [5]:
# db_conf_dict = {
#     "dbname":  os.environ['DB_NAME'],
#     "user":    os.environ['DB_USER'],
#     "password":os.environ['DB_USER_PW'],
#     "host":    os.environ['DB_HOST'],
#     "port": os.environ['DB_PORT']  
# }
# print(db_conf_dict)

## (Option 2) Loading credentials from ENV variables
The env variables can be injected from Kubeflow PodDefault

In [6]:
# !env | grep "SCIVIAS"

In [7]:
'''Uncomment this cell if you are using opton1'''
import sqlalchemy
import psycopg

from applyllm.io import (
    # SqlDBConfig,
    SqlDBHelperFactory,
)
    
db_config = SqlDBHelperFactory.get_db_config_from_env(
    port_key="SCIVIAS_ANALYTICS_DB_PORT"
)
# db_conf_dict = db_conf_obj.to_psycopg_dic(is_analytics=True)
# sqlalchemy_url_obj = db_conf_obj.to_sqlalchemy_url(is_analytics=True)

## Execute SQL with sqlalchemy
* https://docs.sqlalchemy.org/en/20/core/engines.html
* https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#module-sqlalchemy.dialects.postgresql.psycopg

```python
# engine = create_engine('postgresql+psycopg://', creator=connector)
sync_engine = sqlalchemy.create_engine("postgresql+psycopg://", scott:tiger@localhost/test")
```

In [8]:
DEBUG=True

In [9]:
# create sync engine with url_object
# sync_engine = sqlalchemy.create_engine(sqlalchemy_url_obj, echo=DEBUG)
sync_engine = SqlDBHelperFactory.get_sync_engine(db_config=db_config, verbose=DEBUG)

In [10]:
# create table with name test if not exit
# sql1="CREATE TABLE IF NOT EXISTS test (id serial PRIMARY KEY, num integer, data varchar);"

In [11]:
# run_sql(config, sql1)

## Run SQL with results
https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html

In [12]:
from typing import List, Tuple
from collections.abc import Iterable

def print_tuple_list(it: Iterable) -> None:
    for el in it:
        print(*el, sep=",") # print tuple
        

# def run_sql_with_return(engine, sql) -> sqlalchemy.engine.cursor.CursorResult:
def run_sql_with_return(engine, sql) -> Iterable:    
    with engine.connect() as con:
        rs = con.execute(sqlalchemy.text(sql))
        # for row in rs:
        #     print(row)
        return rs

In [13]:
# https://stackoverflow.com/a/24008869
# sql_show_tables="select relname from pg_class;"
sql_show_tables="select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';"

In [14]:
result = run_sql_with_return(sync_engine, sql_show_tables)
print_tuple_list(result)

2024-02-18 11:08:32,291 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-02-18 11:08:32,292 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-18 11:08:32,295 INFO sqlalchemy.engine.Engine select current_schema()
2024-02-18 11:08:32,296 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-18 11:08:32,298 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-02-18 11:08:32,298 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-02-18 11:08:32,303 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-18 11:08:32,304 INFO sqlalchemy.engine.Engine select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';
2024-02-18 11:08:32,304 INFO sqlalchemy.engine.Engine [generated in 0.00091s] {}
2024-02-18 11:08:32,307 INFO sqlalchemy.engine.Engine ROLLBACK
lab_data
test_user_report_info


In [15]:
result = run_sql_with_return(sync_engine, "select count(*) from lab_data")
print_tuple_list(result)

2024-02-18 11:08:32,311 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-18 11:08:32,312 INFO sqlalchemy.engine.Engine select count(*) from lab_data
2024-02-18 11:08:32,312 INFO sqlalchemy.engine.Engine [generated in 0.00099s] {}
2024-02-18 11:08:32,335 INFO sqlalchemy.engine.Engine ROLLBACK
236884


### Create Table
* v2 sqlalchemy DeclarativeBase (ORM) https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#whatsnew-20-orm-declarative-typing
* v2 sqlalchemy ORM quickstart: https://docs.sqlalchemy.org/en/20/orm/quickstart.html
* Create only one table with v2 sqlalchemy ORM: https://stackoverflow.com/questions/19175311/how-to-create-only-one-table-with-sqlalchemy
* v2 upsert https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#orm-enabled-insert-upsert-update-and-delete-statements-with-orm-returning

V1 sqlalchemy (ORM):
* https://www.tutorialspoint.com/sqlalchemy/sqlalchemy_core_creating_table.htm

In [16]:
from sqlalchemy import String, Integer, MetaData, BigInteger
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import Mapped
#from typing_extensions import Annotated
from typing import Optional

# str50 = Annotated[str, 50]

class Base(DeclarativeBase):
    pass
    # type_annotation_map = {
    #     str50: String(50),
    # }

class ReportInfo(Base):
    __tablename__ = "test_user_report_info"
    
    id: Mapped[str] = mapped_column(String(50), primary_key=True, autoincrement=False)
    name: Mapped[str] = mapped_column(String(50))
    age: Mapped[int] = mapped_column(Integer)
    timestamp: Mapped[int] = mapped_column(BigInteger)
    # timestamp: Mapped[int] = mapped_column(Integer)
    
    def __repr__(self) -> str:
        # !r calls repr(self.id) 
        # https://stackoverflow.com/questions/44800801/in-python-format-f-string-strings-what-does-r-mean
        return f"ReportInfo(id={self.id!r}, name={self.name!r}, age={self.age!r}), timestamp={self.timestamp!r}"
    
    def to_dict(self) -> dict:
        return {
            "id": self.id,
            "name": self.name,
            "age": self.age,
            "timestamp": self.timestamp
        }

# table_str=[ReportInfo.__tablename__]

# returns sqlalchemy.Table object
table_objects = [ReportInfo.__table__]  

In [17]:
# Delete table
# ReportInfo.__table__.drop(bind=sync_engine, checkfirst=True)

In [18]:
Base.metadata.create_all(sync_engine, tables=table_objects, checkfirst=True)

2024-02-18 11:08:32,398 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-18 11:08:32,402 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s::VARCHAR AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s::VARCHAR, %(param_2)s::VARCHAR, %(param_3)s::VARCHAR, %(param_4)s::VARCHAR, %(param_5)s::VARCHAR]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s::VARCHAR
2024-02-18 11:08:32,402 INFO sqlalchemy.engine.Engine [generated in 0.00056s] {'table_name': 'test_user_report_info', 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-02-18 11:08:32,406 INFO sqlalchemy.engine.Engine COMMIT


### Postgres Upsert
* v2 upsert https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#orm-enabled-insert-upsert-update-and-delete-statements-with-orm-returning
* https://docs.sqlalchemy.org/en/20/dialects/postgresql.html#postgresql-insert-on-conflict
* https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#orm-enabled-insert-upsert-update-and-delete-statements-with-orm-returning
* https://docs.sqlalchemy.org/en/20/orm/session_transaction.html#session-level-vs-engine-level-transaction-control
* https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#orm-enabled-insert-upsert-update-and-delete-statements-with-orm-returning

session.scalars (iterate over objects)
* https://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session.scalarshttps://docs.sqlalchemy.org/en/20/orm/session_api.html#sqlalchemy.orm.Session.scalars

In [19]:
from sqlalchemy.dialects.postgresql import insert as pgsql_upsert
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
# from sqlalchemy.dialects.postgresql import func
from datetime import datetime
import pytz

current_time = int(datetime.utcnow().timestamp())
# print(type(current_time))
# print(current_time)

stmt = pgsql_upsert(ReportInfo).values(
    [
        {"id": "KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID", "name": "test2", "age": 0, "timestamp": current_time},
        {"id": "KK-SCIVIAS-xxxxx^0054643791^2021-02-03^KIIID", "name": "test", "age": -1, "timestamp": current_time}
    ]
)

# {"id": "KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID", "name": "test", "age": -1, "timestamp": func.now()}

# stmt = pgsql_upsert(ReportInfo).values(timestamp=func.localtimestamp())
# my_values = [
#         {"id": "KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID", "name": "test2", "age": 0},
#         {"id": "KK-SCIVIAS-xxxxx^0054643791^2021-02-03^KIIID", "name": "test", "age": -1}
# ]

# stmt = pgsql_upsert(ReportInfo).values(id="KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID", name="test2", age=0, timestamp=func.localtimestamp())


# stmt = stmt.on_conflict_do_update(
#      index_elements=[ReportInfo.id], set_=dict(id=stmt.excluded.id)
# )

stmt = stmt.on_conflict_do_update(constraint = ReportInfo.__table__.primary_key, set_=stmt.excluded)

# stmt = stmt.on_conflict_do_update(
#      index_elements=[ReportInfo.id], set_=stmt.excluded
# )

# session = sessionmaker(bind=sync_engine)
# with Session() as session: 
# with Session(sync_engine) as session:
# with Session() as session:


"""
It is possible to set the expire_on_commit=False for the session to use the ReportInfo object outside the session,
but this object will not be bound to session and it get outdated in the other sessions.

with Session(sync_engine, expire_on_commit=False) as session:

Reference: https://stackoverflow.com/questions/58291247/using-objects-outside-of-the-sessions-scope-in-sqlalchemy
"""
# with Session(sync_engine, expire_on_commit=False) as session:
with Session(sync_engine) as session:    
    # sqlalchemy.engine.result.ScalarResult
    result = session.scalars(stmt.returning(ReportInfo))
    # result.all() returns a list of ReportInfo objects which is bound to the session
    # unpacking the ReportInfo objects to dictionary to use it outside session as copy
    elements = [e.to_dict() for e in result.all()]    
    session.commit()

2024-02-18 11:08:32,416 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-18 11:08:32,418 INFO sqlalchemy.engine.Engine INSERT INTO test_user_report_info (id, name, age, timestamp) VALUES (%(id_m0)s::VARCHAR, %(name_m0)s::VARCHAR, %(age_m0)s::INTEGER, %(timestamp_m0)s::BIGINT), (%(id_m1)s::VARCHAR, %(name_m1)s::VARCHAR, %(age_m1)s::INTEGER, %(timestamp_m1)s::BIGINT) ON CONFLICT (id) DO UPDATE SET id = excluded.id, name = excluded.name, age = excluded.age, timestamp = excluded.timestamp RETURNING test_user_report_info.id, test_user_report_info.name, test_user_report_info.age, test_user_report_info.timestamp
2024-02-18 11:08:32,419 INFO sqlalchemy.engine.Engine [no key 0.00073s] {'id_m0': 'KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID', 'name_m0': 'test2', 'age_m0': 0, 'timestamp_m0': 1708254512, 'id_m1': 'KK-SCIVIAS-xxxxx^0054643791^2021-02-03^KIIID', 'name_m1': 'test', 'age_m1': -1, 'timestamp_m1': 1708254512}
2024-02-18 11:08:32,421 INFO sqlalchemy.engine.Engine COMMIT


In [20]:
print(elements)

[{'id': 'KK-SCIVIAS-00033^0054643791^2021-02-03^KIIID', 'name': 'test2', 'age': 0, 'timestamp': 1708254512}, {'id': 'KK-SCIVIAS-xxxxx^0054643791^2021-02-03^KIIID', 'name': 'test', 'age': -1, 'timestamp': 1708254512}]


In [21]:
ts = elements[0].get('timestamp', 0)
print(ts)
print(type(ts))

timezone = pytz.timezone('Europe/Berlin')
date_object = datetime.fromtimestamp(ts, tz=timezone)
print(date_object)

1708254512
<class 'int'>
2024-02-18 12:08:32+01:00


In [22]:
# UTC+0 time
date_object = datetime.fromtimestamp(ts)
print(date_object)

2024-02-18 11:08:32
